In [ ]:
import overpy

def get_data(query):
    '''
        Requete sur "http://overpass-api.de/api/interpreter" (url par default)
    '''
    # Obtenir la selection géographique sur oapi serveur français
    # urlFR = u"http://api.openstreetmap.fr/oapi/interpreter"
    api = overpy.Overpass()
    # FIXME pour le département de l'Aisne l'url FR ne réponds un result vide !?
    # api.url = urlFR
    try:
        result = api.query(query)
    except overpy.exception.OverpassUnknownHTTPStatusCode:
        print('UnknownHTTPStatusCode : HTTPS status Code = 500 !')
        result = None
        # raise overpy.exception.OverpassTooManyRequests()
    except overpy.exception.OverpassTooManyRequests:
        print('TooManyRequests : Trop de requêtes pour le serveur Overpass.eu. Patienter et ré-essayer plus tard.')
        result = None
    except overpy.exception.OverpassGatewayTimeout:
        print('TimeOut : Le serveur Overpass.eu ne réponds pas.')
        result = None
    # else :
    # print (type(result))
        # si erreur timeout attendre un moment et recommencer (pas planter)
    return result



def get_osm(departement, musee):
    '''
        Obtenir les objets OSM contenant le tag 'ref:mhs'
        pour un département = '01' par exemple
    '''
    query = "[timeout:900];"
    query_part1 = '''area[admin_level={}]["name"="{}"]->.boundaryarea;
                (node(area.boundaryarea)["ref:mhs"];
                way(area.boundaryarea)["ref:mhs"];
                relation(area.boundaryarea)["ref:mhs"];);'''
    query_end = '''out meta;>;out meta;'''
    for d in departement:
        query = ""
        if 'Miquelon' in d:
            level = '3'
        else:
            level = '6'
        query += query_part1.format(level, d)
        query += query_end
        query = ' '.join(query.replace("\n", "").split())
        # print("Query : ", query)
        logging.debug("log : Osm Query : {}".format(query))
        result = get_data(query)